In [ ]:
#libraries

import rasterio
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import os
import csv
from matplotlib.ticker import MultipleLocator
import math

In [ ]:
# Function to calc relitive depth

# Calibration
def calculate_depth(sum_logbands, slope, intercept):
    
    # Calibration of Sat derived depth
    depth = (slope_cal * sum_logbands) + intercept_cal
    return depth

# Function to load residuals from a CSV file
def load_residuals_from_csv(filename):
    residuals = []
    if os.path.exists(filename):
        with open(filename, 'r') as csvfile:
            csv_reader = csv.reader(csvfile)
            # Skip header
            next(csv_reader)
            # Read data
            for row in csv_reader:
                residuals.append([float(item) for item in row])
    return residuals

# save and re-Save the residuals at each validation observatio
def save_to_residuals(residual_val, depth_cleaned, Lidar_val_cleaned, Ben_val_cleaned, spatial_res, spectral_res, N):
    # Get the next ID by adding 1 to the ID of the last entry in the list, if any
    next_id = Residuals[-1][0] + 1 if Residuals else 1
    # Iterate over each observation point and append its information to the data array
    for i in range(len(residual_val)):
        Residuals.append([next_id + i, residual_val[i], depth_cleaned[i], Lidar_val_cleaned[i], Ben_val_cleaned[i], spatial_res, spectral_res, N])

def save_residuals_to_csv(filename, residuals):
    with open(filename, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        # Write header
        csv_writer.writerow(["ID", "Residual", "Depth", "LiDAR_value", "Benthos_value", "Spatial_resolution", "Spectral_resolution"])
        # Write data
        csv_writer.writerows(residuals)

# Save the metrics of derivation quality
def save_to_results(derivation_technique, satellite_id, bands, slope_cal, intercept_cal, r_squared_val, rmse_val, mean_residual, mean_residual_sand, mean_residual_non_sand):
    # Generate a new ID number by adding 1 to the length of the data array
    new_id = len(results) + 1
    # Append the key information to the data array
    results.append([new_id, derivation_technique, satellite_id, bands, slope_cal, intercept_cal, r_squared_val, rmse_val, mean_residual, mean_residual_sand, mean_residual_non_sand])

In [ ]:
# Define the root directory where the Raw folder is located
root_dir = 'Raw/'

# Define file paths for satellite images
sat_img_dir = os.path.join(root_dir, 'sat_img/Pre_Noise_removal')
PN_path = os.path.join(sat_img_dir, 'PN_image.tif')
PSSD_path = os.path.join(sat_img_dir, 'PSSD_image.tif')
S2_path = os.path.join(sat_img_dir, 'S2_image.tif')

# Define file paths for benthic rasters
benthos_dir = os.path.join(root_dir, 'benthos')
ben120path = os.path.join(benthos_dir, 'ben120.tif')
ben300path = os.path.join(benthos_dir, 'ben300.tif')
ben1000path = os.path.join(benthos_dir, 'ben1000.tif')

# Define file paths for bathy-LiDAR cal val rasters
bathy_LiDAR_dir = os.path.join(root_dir, 'bathy_LiDAR/Tidally_Corrected_with_DEWprofile_200004')
bathy120_cal_path = os.path.join(bathy_LiDAR_dir, 'bathy120_cal.tif')
bathy300_cal_path = os.path.join(bathy_LiDAR_dir, 'bathy300_cal.tif')
bathy1000_cal_path = os.path.join(bathy_LiDAR_dir, 'bathy1000_cal.tif')
bathy120_val_path = os.path.join(bathy_LiDAR_dir, 'bathy120_val.tif')
bathy300_val_path = os.path.join(bathy_LiDAR_dir, 'bathy300_val.tif')
bathy1000_val_path = os.path.join(bathy_LiDAR_dir, 'bathy1000_val.tif')

In [ ]:
# Import sat imagery

# Load PN raster data and set 0 values to NoData (NaN)
with rasterio.open(PN_path) as src_PN:
    PN = src_PN.read().astype(float)  # Read all bands and convert to float
    PN_crs = src_PN.crs
    PN[PN > 10000] = np.nan

# Load PSSD raster data and set 0 values to NoData (NaN)
with rasterio.open(PSSD_path) as src_PSSD:
    PSSD = src_PSSD.read().astype(float)  # Read all bands and convert to float
    PSSD_crs = src_PSSD.crs
    PSSD[PSSD > 10000] = np.nan

# Load S2 raster data and set 0 values to NoData (NaN)
with rasterio.open(S2_path) as src_S2:
    S2 = src_S2.read().astype(float)  # Read all bands and convert to float
    S2_crs = src_S2.crs
    S2[S2 > 10000] = np.nan
    
# Check CRS
print("CRS for PN:", PN_crs)
print("CRS for PSSD:", PSSD_crs)
print("CRS for S2:", S2_crs)

In [ ]:
#Rename those bands

S2b = S2[2]
S2g = S2[1]
S2r = S2[0]

PSSDdb = PSSD[5]
PSSDb = PSSD[4]
PSSDg1 = PSSD[3]
PSSDg2 = PSSD[2]
PSSDy = PSSD[1]
PSSDr = PSSD[0] 

PNdb = PN[3]
PNb = PN[2]
PNg = PN[1]
PNr = PN[0]

In [ ]:
# Plot data as a sanity check,

#visualise the bands
plt.imshow(PSSDg2, cmap='viridis', vmin=200, vmax=750)
plt.colorbar(label='Pixel Value')
plt.show()

#histogram to settle on min and max stretch values
plt.hist(PSSDg2.flatten(), bins =32)

In [ ]:
# Load the benthic rasters and set 15 values to NoData (NaN)
with rasterio.open(ben120path) as src_ben120:
    ben120 = src_ben120.read().astype(float)
    ben120_crs = src_ben120.crs
    ben120[ben120 == 15] = np.nan
    
with rasterio.open(ben300path) as src_ben300:
    ben300 = src_ben300.read().astype(float)
    ben300_crs = src_ben300.crs
    ben300[ben300 == 15] = np.nan
        
with rasterio.open(ben1000path) as src_ben1000:
    ben1000 = src_ben1000.read().astype(float) 
    ben1000_crs = src_ben1000.crs
    ben1000[ben1000 == 15] = np.nan
        
# Check CRS
print("CRS for ben120:", ben120_crs)
print("CRS for ben300:", ben300_crs)
print("CRS for ben1000:", ben1000_crs)

# Plot benthic rasters
plt.imshow(ben120[0], cmap='viridis')
plt.colorbar(label='Pixel Value')
plt.show()

plt.hist(ben120.flatten(), bins =50)

In [ ]:
# Load bathy-LiDAR calibration rasters
with rasterio.open(bathy120_cal_path) as src_bathy120_cal:
    bathy120_cal = src_bathy120_cal.read().astype(float)
    bathy120_cal_crs = src_bathy120_cal.crs
    bathy120_cal[(bathy120_cal == -9999) | (bathy120_cal < 0.5) | (bathy120_cal > 5)] = np.nan
        
with rasterio.open(bathy300_cal_path) as src_bathy300_cal:
    bathy300_cal = src_bathy300_cal.read().astype(float)
    bathy300_cal_crs = src_bathy300_cal.crs
    bathy300_cal[(bathy300_cal == -9999) | (bathy300_cal < 0.5) | (bathy300_cal > 5)] = np.nan

with rasterio.open(bathy1000_cal_path) as src_bathy1000_cal:
    bathy1000_cal = src_bathy1000_cal.read().astype(float)
    bathy1000_cal_crs = src_bathy1000_cal.crs
    bathy1000_cal[(bathy1000_cal == -9999) | (bathy1000_cal < 0.5) | (bathy1000_cal > 5)] = np.nan

# Similarly, load validation data
with rasterio.open(bathy120_val_path) as src_bathy120_val:
    bathy120_val = src_bathy120_val.read().astype(float)
    bathy120_val_crs = src_bathy120_val.crs
    bathy120_val[(bathy120_val == -9999) | (bathy120_val < 0.5) | (bathy120_val > 5)] = np.nan

with rasterio.open(bathy300_val_path) as src_bathy300_val:
    bathy300_val = src_bathy300_val.read().astype(float)
    bathy300_val_crs = src_bathy300_val.crs
    bathy300_val[(bathy300_val == -9999) | (bathy300_val < 0.5) | (bathy300_val > 5)] = np.nan

with rasterio.open(bathy1000_val_path) as src_bathy1000_val:
    bathy1000_val = src_bathy1000_val.read().astype(float)
    bathy1000_val_crs = src_bathy1000_val.crs
    bathy1000_val[(bathy1000_val == -9999) | (bathy1000_val < 0.5) | (bathy1000_val > 5)] = np.nan

# Plot the histogram for bathy1000_val
plt.hist(bathy300_val.flatten(), bins=50, alpha=0.5, label='Validation Data')

# Plot the histogram for bathy1000_cal
plt.hist(bathy300_cal.flatten(), bins=50, alpha=0.5, label='Calibration Data')

# Add labels and legend
plt.xlabel('Pixel Value')
plt.ylabel('Frequency')
plt.legend()

# Show the plot
plt.show()

# Check CRS
print("CRS for bathy120_cal:", bathy120_cal_crs)
print("CRS for bathy300_cal:", bathy300_cal_crs)
print("CRS for bathy1000_cal:", bathy1000_cal_crs)
print("CRS for bathy120_val:", bathy120_val_crs)
print("CRS for bathy300_val:", bathy300_val_crs)
print("CRS for bathy1000_val:", bathy1000_val_crs)

In [ ]:
# open a new dataset
#dont do this if you already have one!!!!
results = []

In [ ]:
#UPDATE ME!!!
# create a variables for the derivation identification
Derivation_Technique = 'Multiband Linear Method'
Satellite_ID = 'PlanetScope SuperDove'
Bands = 'B, G-alt, G'
spatial_res = 3
N = 3 #Number of Bands

# Create a title using the variables
title = f"{Derivation_Technique} on {Satellite_ID} using {Bands} Bands"

# create a variables for the bands used
B1 = PSSDg1
B2 = PSSDg2
B3 = PSSDy
#B4 = PSSDr
#B5 = PSSDb
#B6 = PSSDr

# For the Benthos
Ben = ben300[0]

# And the LiDAR
Lidar_cal = bathy300_cal[0]
Lidar_val = bathy300_val[0]

print(title)

In [ ]:
#UPDATE ME!!!!
# calulate sum_logbands. Take the natural log of the band values and sum them together
sum_logbands = (np.log(B1)) + (np.log(B2)) + (np.log(B3))

# Plot histogram just to analyse sum_logbands now that the data is flattened
plt.figure(figsize=(8, 6))
plt.hist(sum_logbands.flatten(), bins=100, color='blue', alpha=0.7)
plt.xlim(16, 24)
plt.title('Histogram of sum_logbands')
plt.xlabel('sum_logbands')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

# Plot the sum_logbands as an image
plt.figure(figsize=(8, 6))
plt.imshow(sum_logbands, cmap='viridis', vmin=17, vmax=20.5)
plt.colorbar(label='sum_logbands')
plt.title('sum_logbands')
plt.xlabel('Pixel X')
plt.ylabel('Pixel Y')
plt.grid(False)
plt.show()

In [ ]:
#Update MEEE!!!!

#calculate spectral res of derivation
#Spectral Resolution  spectral_res = 10 Log10 [10*N / (spec_range * Avbw )] 
#where  N = no. of bands, Avbw = average band width (nm), spec_range = spectral range of all bands (nm)

#first bandwidth (bw)
PSSDdb_bw = 21 
PSSDb_bw = 50
PSSDg1_bw = 36
PSSDg2_bw = 36
PSSDy_bw = 20
PSSDr_bw = 30
PNdb_bw = 50
PNb_bw = 70
PNg_bw = 60
PNr_bw = 70
S2b_bw = 66
S2g_bw = 36
S2r_bw = 31

#now band mid-points to calc spec_rang
PSSDdb_mp = 441.5
PSSDb_mp = 490
PSSDg1_mp = 531
PSSDg2_mp = 565
PSSDy_mp = 610
PSSDr_mp = 665
PNdb_mp = 425
PNb_mp = 485
PNg_mp = 560
PNr_mp = 655
S2b_mp = 492
S2g_mp = 559
S2r_mp = 664.5

#Update Me!!!
#Calc Spec_Range 
#spec_range = S2r_bw # if only 1 band
spec_range = (PSSDy_13mp + (PSSDy_bw / 2)) - (PSSDg1_mp - (PSSDg1_bw / 2)) #hightest wavelength band first, lowest band last
print("spec_range =", spec_range)

#Update Me!!!
#Calc AvBw
#AvBw = S2r_bw # if 1 band only
AvBw = (PSSDy_bw + PSSDg1_bw + PSSDg2_bw)/N # add all bands
print("average bandwidth =", AvBw)
                                 
#Now calc the spectral res of derivation
spectral_res = 10 * math.log10((10 * N) / (spec_range * AvBw))
print("spectral resolution =", spectral_res)

In [ ]:
# Get the shape of each dataset to ensure we are comparing the same geographic space between all 3 rasters
sum_logbands_shape = sum_logbands.shape
Ben_shape = Ben.shape
Lidar_cal_shape = Lidar_cal.shape

# Print the number of columns and rows in each dataset
print("Shape of sum_logbands:", sum_logbands_shape)
print("Shape of Ben:", Ben_shape)
print("Shape of Lidar_cal:", Lidar_cal_shape)

In [ ]:
# Now plot the relative depth against the real depth
# To do so we need to remove pixels in both array's where either raster has nodata.  
# Find valid pixels where none of the arrays have NaN values
valid_cal_pixels = np.where(~np.isnan(sum_logbands) & ~np.isnan(Ben) & ~np.isnan(Lidar_cal))

# Extract valid pixel values from rasters
Ben_cal_cleaned = Ben[valid_cal_pixels]
Lidar_cal_cleaned = Lidar_cal[valid_cal_pixels]
sum_logbands_cleaned = sum_logbands[valid_cal_pixels]

# Print the length of the cleaned arrays
print("Length of sum_logbands_cleaned:", len(sum_logbands_cleaned))
print("Length of Ben_cal_cleaned:", len(Ben_cal_cleaned))
print("Length of Lidar_cal_cleaned", len(Lidar_cal_cleaned))

In [ ]:
# Plot scatter plot
plt.figure(figsize=(7, 6))
plt.scatter(sum_logbands_cleaned, Lidar_cal_cleaned, s=10, c='b', alpha=0.15)
plt.xlim(np.min(sum_logbands_cleaned)-0.001, np.max(sum_logbands_cleaned)+0.001)  # Set x-axis bounds
plt.ylim(-0.5,8)  # Set y-axis bounds

# Set major and minor gridlines
plt.grid(True, which='major', linestyle='-', linewidth='0.5', color='black')
plt.grid(True, which='minor', linestyle=':', linewidth='0.5', color='black')
plt.gca().yaxis.set_major_locator(MultipleLocator(1.0)) # Set major and minor tick locators
plt.gca().yaxis.set_minor_locator(MultipleLocator(0.5))

# Add labels and title
plt.xlabel('Relative Depth')
plt.ylabel('Depth Observations at Time of Image Capture (m)')
plt.title(title)

# Fit a linear regression line
slope_cal, intercept_cal = np.polyfit(sum_logbands_cleaned, Lidar_cal_cleaned, 1)
predicted = slope_cal * np.array(sum_logbands_cleaned) + intercept_cal
residuals = Lidar_cal_cleaned - predicted
ss_res = np.sum(residuals**2)
ss_tot = np.sum((Lidar_cal_cleaned - np.mean(Lidar_cal_cleaned))**2)
r_squared_cal = 1 - (ss_res / ss_tot)

# Plot the line
plt.plot(sum_logbands_cleaned, slope_cal * sum_logbands_cleaned + intercept_cal, color='red', alpha=1, linewidth=2, linestyle='solid')

# Add the equation of the line with a semi-transparent box
equation_text = f'y = {slope_cal:.4f}x + {intercept_cal:.4f}\n$R^2$ = {r_squared_cal:.2f}'
plt.text(0.95, 0.95, equation_text, horizontalalignment='right', verticalalignment='top', transform=plt.gca().transAxes, fontsize=12, color='black', bbox=dict(facecolor='white', alpha=0.85, edgecolor='grey'))


# Show plot
plt.show()

# Print the important values
print("r2_cal =",r_squared_cal)
print("slope_cal",slope_cal)
print("y-intercept_cal",intercept_cal)

In [ ]:
# Now use the prior function to calibrate the data

depth = calculate_depth(sum_logbands, slope_cal, intercept_cal)

In [ ]:
# Plot depth as an image
plt.figure(figsize=(8, 6))
plt.imshow(depth, cmap='viridis', vmin=0, vmax=6)
plt.colorbar(label='Depth (m)')
plt.title('Satellite Derived Depth')
plt.xlabel('Pixel X')
plt.ylabel('Pixel Y')
plt.grid(False)
plt.show()

In [ ]:
import rasterio
from rasterio.transform import Affine

# Define the output file path
depth_output_path = 'depth.tif'

# Get metadata from one of the input rasters (assuming all have the same metadata)
metadata = src_bathy120_val.meta

# Update metadata for the depth raster
metadata.update({
    'driver': 'GTiff',  # Specify the driver
    'count': 1,  # Number of bands in the output raster
    'dtype': rasterio.float32,  # Data type of the output raster
    'nodata': np.nan,  # NoData value
})

# Create an Affine transformation for the output raster
transform = Affine(src_bathy120_val.transform.a, src_bathy120_val.transform.b, src_bathy120_val.transform.c,
                   src_bathy120_val.transform.d, src_bathy120_val.transform.e, src_bathy120_val.transform.f)

# Write the depth raster to a GeoTIFF file
with rasterio.open(depth_output_path, 'w', **metadata) as dst:
    dst.write(depth.astype(rasterio.float32), 1)  # Write the depth array to the output raster
    dst.transform = transform  # Set the transformation


In [ ]:
#Now validate
# Get the shape of each dataset to ensure we are comparing the same geographic space between all 3 rasters
depth_shape = depth.shape
Ben_shape = Ben.shape
Lidar_val_shape = Lidar_val.shape

# Print the number of columns and rows in each dataset
print("Shape of depth:", depth_shape)
print("Shape of Ben:", Ben_shape)
print("Shape of Lidar_val:", Lidar_val_shape)

In [ ]:
# Now plot the relative depth against the real depth
# To do so we need to remove pixels in both array's where either raster has nodata.  
# Find valid pixels where none of the arrays have NaN values
valid_val_pixels = np.where(~np.isnan(depth) & ~np.isnan(Ben) & ~np.isnan(Lidar_val))

# Extract valid pixel values from rasters
Ben_val_cleaned = Ben[valid_val_pixels]
Lidar_val_cleaned = Lidar_val[valid_val_pixels]
depth_cleaned = depth[valid_val_pixels]

# Print the length of the cleaned arrays
print("Length of depth_cleaned:", len(depth_cleaned))
print("Length of Ben_val_cleaned:", len(Ben_val_cleaned))
print("Length of Lidar_val_cleaned", len(Lidar_val_cleaned))

In [ ]:
#create a validation scatter plot

# Define the output directory to save off to
output_directory = 'scatter_plots/'

# Plot scatter plot
plt.figure(figsize=(7, 6))

# Scatter plot for non-sand (benthic_data == 1)
plt.scatter(depth_cleaned[Ben_val_cleaned == 1], Lidar_val_cleaned[Ben_val_cleaned == 1], s=10, c='darkgreen', alpha=0.35, label='Non-sand')
# Scatter plot for sand (benthic_data == 2)
plt.scatter(depth_cleaned[Ben_val_cleaned == 2], Lidar_val_cleaned[Ben_val_cleaned == 2], s=10, c='orange', alpha=0.35, label='Sand')

plt.xlim(0, 8)  # Set x-axis bounds
plt.ylim(0, 8)  # Set y-axis bounds

# Set major and minor gridlines
plt.grid(True, which='major', linestyle='-', linewidth='0.5', color='black')
plt.grid(True, which='minor', linestyle=':', linewidth='0.5', color='black')
plt.gca().yaxis.set_major_locator(MultipleLocator(1.0)) # Set major and minor tick locators
plt.gca().yaxis.set_minor_locator(MultipleLocator(0.5))

# Add labels and title
plt.xlabel('Satellite Derived Depth (m)')
plt.ylabel('Depth Observations at Time of Image Capture (m)')
plt.title(title)

# Fit a linear regression line
slope_val, intercept_val = np.polyfit(depth_cleaned, Lidar_val_cleaned, 1)
predicted = slope_val * np.array(depth_cleaned) + intercept_val
residuals = Lidar_val_cleaned - predicted
ss_res = np.sum(residuals**2)
ss_tot = np.sum((Lidar_val_cleaned - np.mean(Lidar_val_cleaned))**2)
r_squared_val = 1 - (ss_res / ss_tot)

# Plot the line
plt.plot(depth_cleaned, slope_val * depth_cleaned + intercept_val, color='black', alpha=1, linewidth=1.5, linestyle='solid')

# Calculate RMSE
rmse_val = np.sqrt(np.mean((Lidar_val_cleaned - predicted)**2))

# Add the equation of the line with a semi-transparent box
equation_text = f'y = {slope_val:.3f}x + {intercept_val:.3f}\n$R^2$ = {r_squared_val:.2f}\nRMSE = {rmse_val:.3f}'
plt.text(0.025, 0.97, equation_text, horizontalalignment='left', verticalalignment='top', transform=plt.gca().transAxes, fontsize=12, color='black', bbox=dict(facecolor='white', alpha=0.85, edgecolor='grey'))

# Show legend
#plt.legend()

# Save the plot
plt.savefig(os.path.join(output_directory, f"{title}.png"))

# Show plot
plt.show()

# Print the important values
print("r2_val =", r_squared_val)
print("slope_val =", slope_val)
print("y-intercept_val =", intercept_val)
print("RMSE =", rmse_val)


In [ ]:
# Now  from the observation save each validation observation as well as critical data
# Calculate residuals
residual_val = depth_cleaned - Lidar_val_cleaned

# Separate residuals based on benthic classes
residual_non_sand = [residual_val[i] for i in range(len(residual_val)) if Ben_val_cleaned[i] == 1]
residual_sand = [residual_val[i] for i in range(len(residual_val)) if Ben_val_cleaned[i] == 2]

# Calculate mean residual error for each class
mean_residual = np.mean(residual_val)
mean_residual_non_sand = np.mean(residual_non_sand)
mean_residual_sand = np.mean(residual_sand)

# Print or save the mean residual errors for each class
print("Mean residual error:", mean_residual)
print("Mean residual error for non-sand class:", mean_residual_non_sand)
print("Mean residual error for sand class:", mean_residual_sand)

In [ ]:
# Function to save residuals to a CSV file
# Load previous residuals if they exist
residuals_filename = 'residuals.csv'
Residuals = load_residuals_from_csv(residuals_filename)

# Call the function to save results
save_to_residuals(residual_val, depth_cleaned, Lidar_val_cleaned, Ben_val_cleaned, spatial_res, spectral_res, N)

# Display the results array
print("Residuals:")
for entry in Residuals:
    print(entry)

# Export results to a CSV file
save_residuals_to_csv(residuals_filename, Residuals)

print("Residuals exported to", residuals_filename)

In [ ]:
# Call the function to save overall result metrics
save_to_results(Derivation_Technique, Satellite_ID, Bands, slope_cal, intercept_cal, r_squared_val, rmse_val, mean_residual, mean_residual_sand, mean_residual_non_sand)

# Display the results array
print("Results:")
for entry in results:
    print(entry)
    
# Export results to a CSV file
with open('results.csv', 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    # Write header
    csv_writer.writerow(["ID", "Derivation_Technique", "Satellite_ID", "Bands", "Slope_cal", "Intercept_cal", "R^2_val", "RMSE_val, mean_residual, mean_residual_sand, mean_residual_non_sand"])
    # Write data
    csv_writer.writerows(results)

print("Results exported to results.csv")